# CH09. Push to dockerhub

### 1. Setting docker image to push
Image를 build 했다고, 원격 레지스트리에서 바로 사용가능한 것은 당연히 아니다. 일단 Docker hub의 경우는 build한 image에 대해서 정보를 추가적으로 명시해야 한다. 일단 간단한 에코서버를 구현하여 image를 만들면 다음과 같다.

```shell
 ~/Desktop/Learning-Container/src/echo_service $ docker images
REPOSITORY     TAG          IMAGE ID       CREATED          SIZE
echo_service   latest       8c********0d   14 seconds ago   42.9MB
python         3.8-alpine   76********0d   17 hours ago     42.9MB
```

echo_service는 어떠한 정보도 명시되지 않은 image이므로 먼저 login을 하고 tag를 설정해준다. 최초 login이라면 ID/PW를 입력해야하고, 이는 dockerhub에 대한 계정정보이다.

```shell
 ~/Desktop/Learning-Container/src/echo_service $ docker login
Authenticating with existing credentials...
Login Succeeded
 ~/Desktop/Learning-Container/src/echo_service $ docker tag echo_service leibniz21c/echo_service
 ~/Desktop/Learning-Container/src/echo_service $ docker images
REPOSITORY                TAG          IMAGE ID       CREATED         SIZE
leibniz21c/echo_service   latest       8c********0d   3 minutes ago   42.9MB
echo_service              latest       8c********0d   3 minutes ago   42.9MB
python                    3.8-alpine   76********0d   17 hours ago    42.9MB
 ~/Desktop/Learning-Container/src/echo_service $ 
```

위와 같이 leibniz21c/echo_service라는 이미지가 추가되었고, 이제 dockerhub에 push할 수 있다. 물론 버전명도 명시할 수 있다.
 
```shell
 ~/Desktop/Learning-Container/src/echo_service $ docker tag echo_service leibniz21c/echo_service:1.0.0 
 ~/Desktop/Learning-Container/src/echo_service $ docker images
REPOSITORY                TAG          IMAGE ID       CREATED          SIZE
leibniz21c/echo_service   1.0.0        8c********0d   18 minutes ago   42.9MB
leibniz21c/echo_service   latest       8c********0d   18 minutes ago   42.9MB
echo_service              latest       8c********0d   18 minutes ago   42.9MB
python                    3.8-alpine   76********0d   17 hours ago     42.9MB
$ ~/Desktop/Learning-Container/src/echo_service $ 
```

### 2. Push
이렇게 tag 정보가 명시된 image의 경우는 자신의 dockerhub 원격지에 public 권한으로 push 할 수 있다.
```
 ~/Desktop/Learning-Container/src/echo_service $ docker push leibniz21c/echo_service:1.0.0 
The push refers to repository [docker.io/leibniz21c/echo_service]
11a******d62: Pushed 
f3d******53a: Pushed 
e4b******2d4: Mounted from library/python 
5e*******5fb: Mounted from library/python 
00*******40a: Mounted from library/python 
23********e3: Mounted from library/python 
cb********29: Mounted from library/python 
1.0.0: digest: sha256:00be**********************************************c6c2 size: 1782
 ~/Desktop/Learning-Container/src/echo_service $
```

Mounted from library/python 이라고 되어있는 부분은 중복된 layer가 원격지에 존재하기에 overlay 파일시스템의 원리에 따라 단지 링킹(마운트)가 된다.

### 3. History of image
``` docker history [IMAGE]``` 명령을 통해서 다음과 같이 이미지를 만드는 과정에 대한 정보를 확인할 수 있다.

```shell
 ~/Desktop/Learning-Container/src/echo_service $ docker history leibniz21c/echo_service:1.0.0
IMAGE          CREATED          CREATED BY                                      SIZE      COMMENT
8c********0d   34 minutes ago   CMD ["/bin/sh" "-c" "python3 /app/echo_serve…   0B        buildkit.dockerfile.v0
<missing>      34 minutes ago   ADD echo_server.py /app # buildkit              503B      buildkit.dockerfile.v0
<missing>      34 minutes ago   RUN /bin/sh -c mkdir /app # buildkit            0B        buildkit.dockerfile.v0
<missing>      34 minutes ago   EXPOSE map[50000/tcp:{}]                        0B        buildkit.dockerfile.v0
<missing>      18 hours ago     /bin/sh -c #(nop)  CMD ["python3"]              0B        
<missing>      18 hours ago     /bin/sh -c set -ex;   wget -O get-pip.py "$P…   8.01MB    
<missing>      18 hours ago     /bin/sh -c #(nop)  ENV PYTHON_GET_PIP_SHA256…   0B        
<missing>      18 hours ago     /bin/sh -c #(nop)  ENV PYTHON_GET_PIP_URL=ht…   0B        
<missing>      18 hours ago     /bin/sh -c #(nop)  ENV PYTHON_PIP_VERSION=21…   0B        
<missing>      18 hours ago     /bin/sh -c cd /usr/local/bin  && ln -s idle3…   32B       
<missing>      18 hours ago     /bin/sh -c set -ex  && apk add --no-cache --…   28.8MB    
<missing>      18 hours ago     /bin/sh -c #(nop)  ENV PYTHON_VERSION=3.8.8     0B        
<missing>      2 days ago       /bin/sh -c #(nop)  ENV GPG_KEY=E3FF2839C048B…   0B        
<missing>      2 days ago       /bin/sh -c set -eux;  apk add --no-cache   c…   512kB     
<missing>      2 days ago       /bin/sh -c #(nop)  ENV LANG=C.UTF-8             0B        
<missing>      2 days ago       /bin/sh -c #(nop)  ENV PATH=/usr/local/bin:/…   0B        
<missing>      2 days ago       /bin/sh -c #(nop)  CMD ["/bin/sh"]              0B        
<missing>      2 days ago       /bin/sh -c #(nop) ADD file:80bf8bd014071345b…   5.61MB    
 ~/Desktop/Learning-Container/src/echo_service $
```